prepare datase

In [1]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Functions for downloading and reading MNIST data."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# pylint: disable=unused-import
import gzip
import os
import tempfile

import numpy
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
# pylint: enable=unused-import

import tensorflow as tf
mnist = read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
print (mnist)

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f223b310790>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f223b310810>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f223b3105d0>)


define variables and conditions

In [0]:
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10
n_input = 784
n_classes = 10
dropout = 0.75

define x, y

In [0]:
x = tf.placeholder(tf.float32, [None, n_input])
_X = tf.reshape(x, shape=[-1, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])



shared weights and bias

In [0]:
wc1 = tf.Variable(tf.random_normal([5, 5, 1, 32]))
bc1 = tf.Variable(tf.random_normal([32]))

def conv2d(img, w, b):
  return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1, 1, 1, 1], padding='SAME'),b))

conv1 = conv2d(_X,wc1,bc1)


convolutional layer

In [0]:
def max_pool(img, k):
  return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

conv1 = max_pool(conv1, k=2)


dropout to avoid overfitting

In [0]:
keep_prob = tf. placeholder(tf.float32)
conv1 = tf.nn.dropout(conv1,keep_prob)

second convolutional layer

In [0]:
wc2 = tf.Variable(tf.random_normal([5, 5, 32, 64]))
bc2 = tf.Variable(tf.random_normal([64]))
conv2 = conv2d(conv1,wc2,bc2)
conv2 = max_pool(conv2, k=2)
conv2 = tf.nn.dropout(conv2, keep_prob)


Densely connected layer

In [0]:
wd1 = tf.Variable(tf.random_normal([7*7*64, 1024]))
bd1 = tf.Variable(tf.random_normal([1024]))
dense1 = tf.reshape(conv2, [-1, wd1.get_shape().as_list()[0]])
dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, wd1),bd1))
dense1 = tf.nn.dropout(dense1, keep_prob)

Readout layer

In [0]:
wout = tf.Variable(tf.random_normal([1024, n_classes]))
bout = tf.Variable(tf.random_normal([n_classes]))
pred = tf.add(tf.matmul(dense1, wout), bout)


Testing and training the model

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Launching the session

In [14]:
init = tf.initialize_all_variables()
with tf.Session() as sess:
  sess.run(init)
  step = 1
  while step * batch_size < training_iters:
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob:  dropout})
    if step % display_step == 0:
      acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
      loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
      print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
    step += 1
  print ("Optimization Finished!")
  print ("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))




Instructions for updating:
Use `tf.global_variables_initializer` instead.
Iter 1280, Minibatch Loss= 31807.355469, Training Accuracy= 0.28906
Iter 2560, Minibatch Loss= 17011.226562, Training Accuracy= 0.24219
Iter 3840, Minibatch Loss= 10394.257812, Training Accuracy= 0.46875
Iter 5120, Minibatch Loss= 5458.902344, Training Accuracy= 0.60156
Iter 6400, Minibatch Loss= 9624.007812, Training Accuracy= 0.56250
Iter 7680, Minibatch Loss= 4162.286133, Training Accuracy= 0.71094
Iter 8960, Minibatch Loss= 3302.252930, Training Accuracy= 0.75000
Iter 10240, Minibatch Loss= 3222.419922, Training Accuracy= 0.81250
Iter 11520, Minibatch Loss= 3892.108154, Training Accuracy= 0.78906
Iter 12800, Minibatch Loss= 3230.108643, Training Accuracy= 0.79688
Iter 14080, Minibatch Loss= 3302.411865, Training Accuracy= 0.80469
Iter 15360, Minibatch Loss= 3704.164795, Training Accuracy= 0.84375
Iter 16640, Minibatch Loss= 3784.021973, Training Accuracy= 0.78906
Iter 17920, Minibatch Loss= 4796.196289, Train